In [3]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os

# Add the path to the folder containing ClusterAndPredict
module_path = os.path.abspath(os.path.join('..', '/workspaces/FactChecker-V3'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from ClusterAndPredict.ClusterAndPredict import ClusterAndPredict
from Testing.DataLoader import DataLoader
from Testing.ParameterCreator import ParameterCreator
from Clustering.Helpers.Visualizer import Visualizer

2024-11-15 20:05:32.885111: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 20:05:32.887925: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 20:05:32.896445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 20:05:32.910235: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 20:05:32.914319: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 20:05:32.925875: I tensorflow/core/platform/cpu_feature_gu

: 

In [ ]:
params = ParameterCreator().get_parameters()
results = []
cluster_dfs = []
print("Number of experiments to run: ", len(params))
for param in params:
    percentage = 0.75
    data_loader = DataLoader(percentage, True, param['random_seed'])
    use_only_card = param['use_only_CARD']
    size_of_dataset = param['size_of_dataset']
    del param['size_of_dataset']
    del param['use_only_CARD']

    train_df, test_df = data_loader.create_train_test_df(False, False, False, True, size_of_dataset)
    if use_only_card:
        print("using card")
        train_df, test_df = data_loader.create_train_test_df(True, True, True, True, size_of_dataset)


    clf = ClusterAndPredict(**param, train_df=train_df)
    clf.fit(test_df['Text'].tolist(), test_df['Numerical Rating'].tolist())
    # Print best parameters
    best_estimator = clf
    score = best_estimator.score([], [])
    object_output = best_estimator.get_all_performance_metrics()
    cluster_df = object_output['cluster_df']
    cluster_dfs.append(cluster_df)
    output = {
        'percentage': percentage,
        'score': score,
        'accuracy': best_estimator.get_accuracy(),
        'was_supervised_umap_used': best_estimator.get_was_supervised(),
        'metrics': best_estimator.get_all_performance_metrics(),
        'size_of_dataset': size_of_dataset,
        'use_only_CARD': use_only_card
    }
    # Prepend the value 'param' to the keys in params
    for key, value in param.items():
        local_key = 'params.' + key
        output[local_key] = value

    results.append(output)
    results_df = pd.json_normalize(results)
    results = []


All data sources are being used
Number of experiments to run:  1
Numerical Rating
3    3256
1      28
Name: count, dtype: int64
Numerical Rating
3    771
1     51
Name: count, dtype: int64
using card
All data sources are being used
Numerical Rating
3    4412
1    2474
Name: count, dtype: int64
Numerical Rating
3    893
1    828
Name: count, dtype: int64
Fitting
Getting embeddings...
Getting embeddings for batch  0  out of  1727
Getting embeddings for batch  50  out of  1727
Getting embeddings for batch  100  out of  1727
Getting embeddings for batch  150  out of  1727
Getting embeddings for batch  200  out of  1727
Getting embeddings for batch  250  out of  1727
Getting embeddings for batch  300  out of  1727
Getting embeddings for batch  350  out of  1727
Getting embeddings for batch  400  out of  1727
Getting embeddings for batch  450  out of  1727
Getting embeddings for batch  500  out of  1727
Getting embeddings for batch  550  out of  1727
Getting embeddings for batch  600  out of

In [ ]:
results_df = pd.json_normalize(results)
results_df